<a href="https://colab.research.google.com/github/vipul43/project_MASK/blob/main/basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# importing dataset from kaggle
!pip install -q kaggle &> /dev/null
!mkdir ~/.kaggle &> /dev/null
!echo '{"username":"saivipul","key":"f4e9e153799bf7289426d9655d02fdca"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json &> /dev/null
# enter dataset API command below
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset &> /dev/null
!unzip face-mask-12k-images-dataset.zip &> /dev/null
!rm -rf face-mask-12k-images-dataset.zip &> /dev/null

# dataset is stored in 'Face Mask Dataset' folder

In [3]:
# Paths
# train images paths
train_images_with_mask_path = '/content/Face Mask Dataset/Train/WithMask/'
train_images_without_mask_path = '/content/Face Mask Dataset/Validation/WithoutMask/'
# dev images paths
dev_images_with_mask_path = '/content/Face Mask Dataset/Validation/WithMask/'
dev_images_without_mask_path = '/content/Face Mask Dataset/Validation/WithoutMask/'
# test images paths
test_images_with_mask_path = '/content/Face Mask Dataset/Test/WithMask/'
test_images_without_mask_path = '/content/Face Mask Dataset/Test/WithoutMask/'

In [30]:
# loading entire images dataset
# label 0: WithMask
# label 1: WithoutMask
batch_size = 16

import numpy as np
from keras.preprocessing.image import load_img, img_to_array, array_to_img
import tensorflow as tf

# loading train images dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/Face Mask Dataset/Train',
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(64, 64),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)
print(type(train_dataset))

# loading validation images dataset
dev_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/Face Mask Dataset/Validation',
    labels="inferred",
    label_mode="binary",
    class_names=None,
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(64, 64),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)
print(type(dev_dataset))

# format of data in the dataset
# for image, label in train_dataset:
#   print(image)
#   print(label)
#   break

Found 10000 files belonging to 2 classes.
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
Found 800 files belonging to 2 classes.
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


In [32]:
# MODEL IS HERE!!!

from keras import Input
from keras.layers import Conv2D, Flatten, Dense
from keras.models import Model
from keras.utils import plot_model

def make_model(input_shape):
  inputs = Input(shape=input_shape)
  x = Conv2D(
      filters=4,
      kernel_size=8,
      strides=(1, 1),
      padding="valid",
      data_format=None,
      dilation_rate=(1, 1),
      groups=1,
      activation='relu',
      use_bias=True,
      kernel_initializer="glorot_uniform",
      bias_initializer="zeros",
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  )(inputs)
  x = Flatten()(x)
  x = Dense(
      128,
      activation='relu',
      use_bias=True,
      kernel_initializer="glorot_uniform",
      bias_initializer="zeros",
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  )(x)
  outputs = Dense(
      1,
      activation='sigmoid',
      use_bias=True,
      kernel_initializer="glorot_uniform",
      bias_initializer="zeros",
      kernel_regularizer=None,
      bias_regularizer=None,
      activity_regularizer=None,
      kernel_constraint=None,
      bias_constraint=None,
  )(x)

  model = Model(inputs=inputs, outputs=outputs)
  model.summary()
  plot_model(model, to_file='model_basic_with_updated_conv_and_dense_layers.png')
  return model

In [35]:
# Getting model and training
import keras.backend as K

model = make_model((64, 64, 1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
K.set_value(model.optimizer.lr, 0.001) # tweaking hyperparameter
model.fit(train_dataset, epochs=10, steps_per_epoch=len(train_dataset), verbose=1)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 57, 57, 4)         260       
_________________________________________________________________
flatten_3 (Flatten)          (None, 12996)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               1663616   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 1,664,005
Trainable params: 1,664,005
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 47s 74ms/step - loss: 42.6791 - accuracy: 0.7814
Epoch 2/10
6

In [39]:
# Making predictions - all at a time
# More accurate metrics - error

def getError(model):
  error = 0
  for image, label in dev_dataset:
    prediction = model.predict(image, verbose=0)
    for i in range(len(prediction)):
      if(label[i]!=prediction[i]):
        error+=1
  error/=(len(dev_dataset)*batch_size)
  return (error)

  getError(model)